In [ ]:
import pandas as pd
  # 데이터 처리 모듈

import numpy as np
import math
import missingno as msno
from scipy.stats import norm
  # 행렬, 선형대수 등 통계 연산 모듈

from google.colab import drive
drive.mount("/content/drive")

import seaborn as sns 
import matplotlib.pyplot as plt
  # 데이터 시각화 모듈

from typing import *
  # 타입 어노테이션 모듈(dynamic -> static)

from sklearn.preprocessing import *
from sklearn.decomposition import *
from sklearn.feature_selection import *
from sklearn.impute import *

!pip install mlxtend

Mounted at /content/drive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

### 데이터 분석 문제의 소재

* 온라인 커머스 플랫폼 DS로 취직한 당신의 첫 프로젝트
* 작년 대히트였던 배송비 무료 쿠폰 이벤트 시즌2 진행
* 비상장기업이었던 작년과 달리 올해는 IPO공개 후 상장하여 좀 더 스마트하게 쿠폰을 뿌리고자 하는 경영진
* 어떻게 쿠폰을 지급하고 이벤트를 진행할 것인가.

### SY's Q.

* 나이, 성별, 지역에 따른 군집 사이에 checkout, monthly_spend, shipping_fee, coupon, subscriber변수에 대한 통계적으로 유의미한 차이가 있는지 검토

| 고객 | 비회원(guest) / 일반 회원, 연간(유료)회원으로 구매 가능 |  |
| --- | --- | --- |
|  | 연간 회원(subscriber) | 무조건 배송비 무료 |
|  | 비회원(guest) / 일반 회원 | 일정 금액 이상 구매 시 배송비 무료 |
| 배송비 무료 금액 | 배송지역에 따라 다름 | 도서산간 / 해외 등 |
| 해외 배송비 | 배송량에 따라 상이 |  |
| 배송비 무료 쿠폰 정책 | 배송비가 있고 배송비 무료 쿠폰을 보유한 경우 자동 적용 후 소멸 | 1회만 사용가능 |

| feature | type | details |
| --- | --- | --- |
| age | integer | 행사 당시 유저 나이 |
| basket | float | 장바구니 총액(단위:천 원) |
| checkout | boolean(TF) | 결제(최종 구매) 여부 |
| region | string(object) | 배송지역 코드 |
| coupon | boolean | 배송비 무료 쿠폰 보유 여부 |
| gender | string | 유저성별 M F |
| monthly_spend | float | 행사 당시 월평균 결제금액(단위: 천 원) |
| shipping_fee | float | 배송비 |
| subscriber | boolean | 연간 회원 여부 |
| tenure | integer | 행사 당시 유저 가입 기간(단위: 월) |

* 출처 : 
  
  https://github.com/JaydenRyou/Manual-Implementation-of-K-Means-Clustering-Algorithm

  https://github.com/tofti/python-kmeans/blob/master/kmeans.py

In [ ]:
# 군집분석 with 사이킷런

'''
scaler = StandardScaler().fit(dataframe)
dataframe_scaled = scaler.transform(dataframe)

Kmeans = KMeans(
  n_clusters = 3,
  init = "random",
  max_iter = 300,
  n_init = 10,
  random_state = 2023
)

y_Kmeans = Kmeans.fit_predict(dataframe_scaled)
'''

In [ ]:
# 군집분석 with for loop
# 과연 최적의 k값을 어떻게 찾을 것인가
# 데이터 포인트와 소속 클러스터의 중심값 사이의 거리 합을 최소화하는 기준


# 1. 데이터에서 임의의 k점을, 초기화 클러스터 중심으로 선택 : initializing the centroids
# 2. 모든 점에서 k클러스터 중심값으로부터의 거리를 계산(거리: 유클리디안) : assigning data points to clusters
def L2distance(xi, xj):
  return sum((xi - xj)**2) / len(xi)

def groupCenter(x):
  x = np.array(x)
  return x.mean(axis = 0)


# 3. 모든 점이 가장 근접한 클러스터 중심에 할당된 다음 산술평균을 활용하여 클러스터 중심값을 다시 산출 : assigning data points to clusters
def KmeansClustering(x, k, seed: int = 2023, iterable_num: int = 25):
  logs: List = []
  np.random.seed(seed)
  centers = x[
      np.random.choice(
          len(x),
          size = k,
          replace = False
      )
        # X벡터의 길이, 군집의 개수 k, 비복원추출
  ] 
    # x벡터 넘파이 배열의 "리스트 컴프리핸션으로 선언한 인덱스" 번호에 해당하는 원소를 중심값으로 선언
  
  for it in range(iterable_num):
    group: tuple = {}

    for index in range(k):
      group[index]: List = []

  # 데이터마다 근접한 클러스터 중심값 탐색
    for row in x:
      temp: List = []
      for index in range(k):
        temp.append(L2distance(centers[index], row))
          # 함수 적용
      group[np.argmin(temp)].append(row.tolist())

    for index in range(k):
      group_temp = np.array(group[index])
      group_temp = np.c_[
          group_temp, 
          np.full(len(group_temp), index)
      ]
      if index == 0:
        grouped = group_temp
      else:
        grouped = np.append(
            grouped, group_temp, axis = 0
        )

    # 중심값을 centers_new 리스트에 update
      centers_new: List = []
      for index in range(k):
        centers_new.append(groupCenter(group[index]).tolist())
          # 함수 적용
      centers_new = np.array(centers_new)

      # if updated center == center, break
      if np.sum(centers - centers_new) == 0:
        break
      else:
        centers = centers_new
        logs.append(grouped)
          
  return grouped, logs, it

In [ ]:
class Clustering():

  def __init__(self, data_commerce = None, data_current_users = None,
               centroid = None, cluster = None):
    self.data_commerce = data_commerce
    self.data_current_users = data_current_users
    self.centroid = centroid
    self.cluster = cluster
      
  
  def importData(self, path_data_commerce: str, path_data_current_users: str):
    data_commerce = pd.read_csv(
        path_data_commerce, 
        index_col = False,
        na_values = "NaN"
    )
      # "/content/drive/MyDrive/HanaFinanceDT/gooppang.csv"
    data_current_users = pd.read_csv(
        path_data_current_users,
        index_col = False,
        na_values = "NaN"
    )
      # "/content/drive/MyDrive/HanaFinanceDT/current_users.csv"
    
    data_commerce["gender"] = np.where(data_commerce["gender"] == "f", 1, 0)
      # 1이면 여성, 0이면 남성

    print(data_commerce.info())
    print(data_current_users.info())

    print(data_commerce.groupby(["subscriber"]).size())
    print(data_commerce.groupby(["checkout"]).size())
      # age, region, gender 변수와 
      # monthly_spend, subscriber, checkout, tenure 변수 사이에 패턴이 있는지 EDA
    
    self.data_commerce = data_commerce
    self.data_current_users = data_current_users


# Elbow Method for finding the Optimal Number of Cluster in Kmeans
# Select the number of clusters for the dataset : K
# Within Cluster Sum of Square(WCSS)
  @property
  def calculateCost(self, X, centroid, cluster):
    sum: int = 0
    for index, value in enumerate(X):
      sum += np.sqrt(
          ( centroid[int(cluster[index]), 0] - value[0] )**2 + \
          ( centroid[int(cluster[index]), 1] - value[1] )**2
      )
        # euclidean distance
    
    return sum
    

  @calculateCost.setter
  def Kmeans(self, data: pd.DataFrame, X, k: int):
    difference = 1
      # 모든 점에서 k클러스터 중심값으로부터의 거리(distance of the point form all centroids)
    cluster = np.zeros(X.shape[0])
    centroid = data.sample(n = k).values
      # Q&A
    
    while difference:
      for index, row in enumerate(X):
        mn_distance = float("inf")

        for idx, centroid in enumerate(centroid):
          similarity = np.sqrt(
              (centroid[0] - row[0])**2 + (centroid[1] - row[1])**2
          )
            # L2 거리유사도 정의

          if mn_distance > similarity:
            mn_distance = similarity
            cluster[index] = idx
          
          new_centroid = pd.DataFrame(X).groupby(
              by = cluster
          ).mean().values
      

          if np.count_nonzero(centroid - new_centroid) == 0:
            difference = 0
          else:
            centroid = new_centroid

      self.centroid = centroid
      self.cluster = cluster

    return self.centroid, self.cluster


  def elbowMethodFindK(self, data: pd.DataFrame, X, k: int):
    cost_list: List = []

    for k in range(1, 10, 1):
      centroids, cluster = Kmeans(data, X, k)
    cost = calculateCost(X, centroids, cluster)
    cost_list.append(cost)


In [ ]:
data_commerce

,age,basket,checkout,region,coupon,gender,monthly_spend,shipping_fee,subscriber,tenure
0,45.0,29.5,False,B,True,1,103.21,20.0,False,14.0
1,41.0,40.9,False,B,False,0,118.34,20.0,False,2.0
2,30.0,41.5,False,B,False,1,144.90,20.0,False,31.0
3,34.0,20.8,False,B,True,1,85.79,20.0,False,1.0
4,42.0,24.6,False,B,False,0,91.32,20.0,False,5.0
...,...,...,...,...,...,...,...,...,...,...
59995,34.0,28.2,False,C,False,1,101.73,10.0,False,2.0
59996,25.0,38.9,False,B,True,1,115.61,20.0,False,4.0
59997,40.0,24.0,False,B,False,1,112.53,20.0,False,6.0
59998,52.0,37.6,False,B,False,0,129.42,20.0,False,8.0


연관규칙 마이닝

* 출처: https://hands-on.cloud/implementation-of-fp-growth-algorithm-using-python/

In [ ]:
# 신뢰도(confidence) = P(A∩B) / P(A)
# 지지도(support) = P(A∩B)
# 향상도(lift) = P(B|A) / P(B) = 지지도 / 신뢰도


### 기본적인 의문점 탐구 → 연습문제

* 배송지역별 배송료 무료 금액의 적용 전후?
* 쿠폰 적용 전후?
* 연간 회원은 무조건 무료? 아니면 배송비 적용 후 할인?
* 장바구니 총액(basket)은 배송비(shipping_fee)가 포함된 것인지?

그외의 다른 의문점은?

앞으로의 분석을 위한 불가피한 가정은?